In [ ]:
import utils
import flower_photos_data
import vgg16
import tensorflow as tf
import numpy as np
from os.path import isfile, isdir
import os

In [ ]:
label_dict ={0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}

In [ ]:
def pre_train_data():
    if not isdir( 'data/flower_photos'):
        if not isfile('data/flower_photos.tar.gz'):
            with utils.DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
                urlretrieve(download_url,zip_file,pbar.hook)
        with tarfile.open('data/flower_photos.tar.gz') as tar:
           tar.extractall(path='data/')
  
    batch_size = 10
    codes = []
    labels= []
    
    input_ = tf.placeholder(dtype=tf.float32, shape=[None, 224, 224, 3] )
    vgg = vgg16.Vgg16()
    
    with tf.Session() as sess:
        vgg.build_nn(input_)
        for index,type in enumerate(os.listdir( 'data/flower_photos')):
            path = os.path.join( 'data/flower_photos', type)
            if not isdir(path):
                continue
            print('Loading {} images .......'.format(type))
            files = os.listdir(path)
            images=[]
            for i,file in enumerate(files,1):
                file = os.path.join( 'data/flower_photos',type, file)
                image = utils.load_image(file)
                image = utils.resize_image(image)
                images.append(image)
                labels.append(index)
                if i % batch_size==0 or i==len(files):
                    batch = np.asarray(images)
                    batch_codes = sess.run(vgg.relu6, feed_dict={input_:batch})
                    codes.append(batch_codes)
                    print('{} images processed'.format(ii))
    codes = np.asarray(codes)
    print(codes.shape)
    labels = np.array(labels)
    print(labels.shape)
    codes, labels = utils.shuffle_data(codes, labels)       
    labels_onehot = utils.one_hot_encode(labels)            
    train_codes, train_labels, valid_codes, valid_labels, test_codes, test_labels = \
        utils.split_data(codes, labels_onehot, train_size=0.8, valid_size=0.1, test_size=0.1)
    os.mkdir('data/flower_pre_train_npy')
    np.save('data/flower_pre_train_npy/train_codes.npy', train_codes)
    np.save('data/flower_pre_train_npy/train_labels.npy', train_labels)
    np.save('data/flower_pre_train_npy/valid_codes.npy', valid_codes)
    np.save('data/flower_pre_train_npy/valid_labels.npy', valid_labels)
    np.save('data/flower_pre_train_npy/test_codes.npy', test_codes)
    np.save('data/flower_pre_train_npy/test_labels.npy', test_labels)    

In [ ]:
if not isdir('data/flower_pre_train_npy'):
    process_data()
train_codes = np.load('data/flower_pre_train_npy/train_codes.npy')
train_labels = np.load('data/flower_pre_train_npy/train_labels.npy')
valid_codes = np.load('data/flower_pre_train_npy/valid_codes.npy')
valid_labels = np.load('data/flower_pre_train_npy/valid_labels.npy')
print("Train codes: {}, labels: {}".format(train_codes.shape, train_labels.shape))
print("Valid codes: {}, labels: {}".format(valid_codes.shape, valid_labels.shape))

In [ ]:
inputs_ = tf.placeholder(dtype=tf.float32, shape=[None, codes.shape[1]], name='inputs')
labels_ = tf.placeholder(dtype=tf.float32, shape=[None,len(label_dict)], name='labels')

In [ ]:
hidden = tf.contrib.layers.fully_connected(inputs_, 256, activation_fn=tf.nn.relu)
logits = tf.contrib.layers.fully_connected(hidden, labels_vecs.shape[1], activation_fn=None)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
prediction = tf.nn.softmax(logits)
compare = tf.equal(tf.argmax(prediction,axis=1), tf.argmax(labels_, axis=1))
accuracy = tf.reduce_mean(tf.cast(compare, dtype=tf.float32))
optimizer = tf.train.AdamOptimizer().minimize(cost)
saver = tf.train.Saver()

In [ ]:
epoches = 10
iteration = 0
predict =[]

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Start training ...... ')
    for e in range(epoches):
        for batch_codes, batch_labels in utils.get_batches(train_codes, train_labels, n_batches=10):
            iteration += 1
            loss,  _ = sess.run([cost,  optimizer],
                               feed_dict={inputs_:batch_codes, labels_:batch_labels})
            print("Epoch: {}/{}".format(e+1, epoches),
                  "Iteration: {}".format(iteration),
                  "Train loss: {:.5f}".format(loss))
            if iteration%10==0:
                val_acc, predict = sess.run([accuracy, prediction], feed_dict={inputs_:valid_codes, labels_:valid_labels})
                print("-------------------")
                print("Epoch: {}/{}".format(e+1, epoches),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
                print("-------------------")

    saver.save(sess, "data/flower_pre_train_checkpoints/flowers.ckpt")

In [ ]:
test_codes = np.load('data/flower_pre_train_npy/test_codes.npy')
test_labels = np.load('data/flower_pre_train_npy/test_labels.npy')
print("Test codes: {}, labels: {}".format(test_codes.shape, test_labels.shape))

In [ ]:
with tf.Session() as sess:
    print('Restoring parameters ......')
    saver.restore(sess, tf.train.latest_checkpoint('data/flower_pre_train_checkpoints'))
    predict, acc = sess.run([prediction, accuracy], feed_dict={inputs_:test_codes, labels_:test_labels, keep_prob:1.0})
    print(acc)